# Tensorboard

In [1]:
import tensorflow as tf
import numpy as np
tf.compat.v1.reset_default_graph()

x = np.linspace(0, 2*np.pi, 100)
y = np.cos(x)

summary_writer = tf.summary.create_file_writer('tensorboard_log')

with summary_writer.as_default():
    for i, j in enumerate(y):
        tf.summary.scalar('cos', j, step=i)

In [15]:
import tensorflow as tf
import datetime
tf.compat.v1.reset_default_graph()


mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])


model = create_model()

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()


train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

@tf.function
def train_step(x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    train_accuracy(y_train, predictions)

@tf.function
def test_step(x_test, y_test):
    predictions = model(x_test)
    loss = loss_object(y_test, predictions)

    test_loss(loss)
    test_accuracy(y_test, predictions)


current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs4/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs4/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
EPOCHS = 5

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_dataset:
        train_step(x_train, y_train)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for (x_test, y_test) in test_dataset:
        test_step(x_test, y_test)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))

    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()



Epoch 1, Loss: 0.24555976688861847, Accuracy: 92.85333251953125, Test Loss: 0.122795969247818, Test Accuracy: 96.1500015258789
Epoch 2, Loss: 0.10463278740644455, Accuracy: 96.94000244140625, Test Loss: 0.0920180231332779, Test Accuracy: 97.1199951171875
Epoch 3, Loss: 0.07288003712892532, Accuracy: 97.75, Test Loss: 0.07447461783885956, Test Accuracy: 97.57999420166016
Epoch 4, Loss: 0.05563836172223091, Accuracy: 98.21333312988281, Test Loss: 0.06841357052326202, Test Accuracy: 97.93000030517578
Epoch 5, Loss: 0.045263029634952545, Accuracy: 98.57666778564453, Test Loss: 0.06428424268960953, Test Accuracy: 98.0999984741211


# Tensorboard (keras)

In [3]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="fit_logs/", histogram_freq=1)

model.fit(x=x_train,
          y=y_train,
          epochs=5,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

Epoch 1/5
   1/1875 [..............................] - ETA: 0s - loss: 2.3653 - accuracy: 0.1562WARNING:tensorflow:From C:\Users\user\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2201 - accuracy: 0.9345 - val_loss: 0.1269 - val_accuracy: 0.9612
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0967 - accuracy: 0.9700 - val_loss: 0.0745 - val_accuracy: 0.9774
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0696 - accuracy: 0.9784 - val_loss: 0.0727 - val_accuracy: 0.9762
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0519 - accuracy: 0.9835 - val_loss: 0.0661 - val_accuracy: 0.9802
Epoch 5/5
1875/1875 [==============================]